
Format of input zip file:
- zip_file/
    - Images without Labels/
       - sample_name1.jpg
       - sample_name2.jpg
       - sample_name3.jpg
    - Raw Data.csv

RawData.csv should have teh following columns:
1. "TID" --> Track Id
2. "PID" --> Picture Id/Frame Number
3. "x [pixel]"" -->  x coordinate
4. "y [pixel]"  --> y coordinate
5. "subpopulation" -->containing "N"/"L"/"M"/"H" for subpopulations

Expected Directory Structure after this code:
MEMTRack/
   - data/
      - zip file
        - videomap.txt
        - collagen/
            - video55/
                - frame1/
                    - /annotations_motility_low
                      - 1.txt
                      - 2.txt
                    - /annotations_motility_wiggle
                    - /annotations_motility_mid
                    - /annotations_motility_high
                    - bacteria/
                       - 1/
                          - xy_coord/
                             - 0.txt
                             - 1.txt
                        - 2/
                    - images/
                       - 0.tif
                       - 1.tif
                   
            - video56/


In [1]:
%config Completer.use_jedi = False

In [2]:
import zipfile
import openpyxl
import os
import csv
import cv2
import shutil
import PIL
import glob
import pandas as pd
import numpy as np
from natsort import natsorted
from PIL import Image

# Loading New Data

Update the variables in the next cell according to your directory structure

In [3]:
zip_data_dir_path = "/home/medha/BacteriaDetectionTracking/MEMTrack/" # path to zip file
zip_dir = "Control_4.zip" # name of zip file

final_data_dir = data_dir = "/home/medha/BacteriaDetectionTracking/MEMTrack/data/" # path to final data folder

target_data_sub_dir = "/home/medha/BacteriaDetectionTracking/MEMTrack/data/collagen/" # path to sub directory in main data folder
os.makedirs(target_data_sub_dir, exist_ok=True)

#videomap.txt: containing video number and video name. Update sample_videomap.txt in data/. WIll be updated on its own
video_map_path = "/home/medha/BacteriaDetectionTracking/MEMTrack/data/videomap.txt" #path to video map

csv_file_name = "Raw Data.csv"

In [4]:
# unzip data
zip_data_dir = os.path.join(zip_data_dir_path, zip_dir)
with zipfile.ZipFile(zip_data_dir, 'r') as zip_ref:
    zip_ref.extractall(zip_data_dir_path)

In [5]:
#find video number from video map
def find_next_video_num(data_dir):
    #data_dir = "../Data/"
    data_sub_dirs = glob.glob(f'{data_dir}/*')  
    print("data dirs: ", data_sub_dirs)
    video_dirs = glob.glob(f'{data_dir}/*/*')  
    max_id = 0
    for video in video_dirs:
        #print(video)
        if "ipynb" not in video:
            video_id = int(video.split("video")[1])
            #print(video_id)
            max_id = max(video_id, max_id)
    print("last video num: ",max_id)
    return max_id +1

In [6]:
video_num = find_next_video_num(data_dir)
print("curr video num: ",video_num)

data dirs:  ['/home/medha/BacteriaDetectionTracking/MEMTrack/data/videomap.txt', '/home/medha/BacteriaDetectionTracking/MEMTrack/data/sample_videomap.txt']
last video num:  0
curr video num:  1


In [7]:
video_dir = os.path.join(zip_data_dir_path, zip_dir.split(".zip")[0])

In [8]:
video_dir

'/home/medha/BacteriaDetectionTracking/MEMTrack/Control_4'

In [9]:
# update video mapping text file?
# can have an argument--> if we need to or not? or manual
video_map = open(video_map_path,'a+')
video_strain = "_".join(video_dir.strip().rsplit("/",3)[1:-1] if video_dir[-1] is "/" else  video_dir.strip().rsplit("/",2)[1:])
video_map.write("\n")
video_map.write(f"{video_num}, {video_strain}")
video_map.close()

<>:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_58527/3806333217.py:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
  video_strain = "_".join(video_dir.strip().rsplit("/",3)[1:-1] if video_dir[-1] is "/" else  video_dir.strip().rsplit("/",2)[1:])


In [10]:
#confirming image source
count = 0
images_source = video_dir + "/Images without Labels/"
files = natsorted(os.listdir(images_source))
files[:5]

['Control Set 1_b0s2t6450c0x3255-660y0-492.tif',
 'Control Set 1_b0s2t6451c0x3255-660y0-492.tif',
 'Control Set 1_b0s2t6452c0x3255-660y0-492.tif',
 'Control Set 1_b0s2t6453c0x3255-660y0-492.tif',
 'Control Set 1_b0s2t6454c0x3255-660y0-492.tif']

create image data with annotations easy and hard

In [11]:
data_dir = video_dir + "/"
all_files = os.listdir(data_dir)
#csv_file_name = list(filter(lambda f: f.endswith('Tracked.csv'), all_files))[0] if list(filter(lambda f: f.endswith('.csv'), all_files)) else list(filter(lambda f: f.endswith('.xlsx'), all_files))[0]
#csv_file_name = "Raw Data.csv"
csv_file = data_dir + csv_file_name
# csv_file = source + "/" + video_source + "/" + frame_source + "/" + "1.csv"
print(csv_file_name)
if ".csv" not in csv_file_name:
    read_file = pd.read_excel(csv_file, engine='openpyxl')
    read_file.to_csv (csv_file.split("xlsx")[0]+".csv", 
                  index = None, header=True)
    csv_file = csv_file.split("xlsx")[0]+".csv"
print(csv_file)
file = open(csv_file)
csvreader = csv.reader(file)
header = []
header = next(csvreader)
# header = header[0].split("\t")
print(header)
header = [x.lower() for x in header]
img_id = header.index('pid') #  header.index('t [Frame]') 
x_id = header.index('x [pixel]')
y_id = header.index('y [pixel]')
try:
    sticking_id = header.index('sticking')
    subpop_id = header.index('subpopulation')
except: 
    sticking_id = None
    subpop_id = None
try:
    diff_id = header.index('e/h')
    
except:
    diff_id = None
    print("No diff id found, assuming all easy")
print(img_id)
print(x_id)
print(y_id)
print(sticking_id)
print(subpop_id)

rows = []
for row in csvreader:
        rows.append(row)
#rows
file.close()

Raw Data.csv
/home/medha/BacteriaDetectionTracking/MEMTrack/Control_4/Raw Data.csv
['Nr', 'TID', 'PID', 'x [pixel]', 'y [pixel]', 't [sec]', 'I [val]', 'Len [pixel]', 'D2S [pixel]', 'D2R [pixel]', 'D2P [pixel]', 'v [pixel/sec]', '? [deg]', '?? [deg]', 'E/H', 'DEL_Diff', 'Sticking', 'Subpopulation']
2
3
4
16
17


In [12]:
count = 0
dest_dir = target_data_sub_dir + f"/video{video_num}/"
images_target =dest_dir + "/frame1/images/"
annotations_easy = dest_dir + "/frame1/annotations_easy/"
annotations_easy_hard = dest_dir + "/frame1/annotations_easy_hard/"
annotations_hard = dest_dir + "/frame1/annotations_hard/"
annotations_very_hard = dest_dir + "/frame1/annotations_veryhard/"
annotations_easy_hard_veryhard = dest_dir + "/frame1/annotations_easy_hard_veryhard/"

annotations_motility_high = dest_dir + "/frame1/annotations_motility_high/"
annotations_motility_low = dest_dir + "/frame1/annotations_motility_low/"
annotations_motility_wiggle = dest_dir + "/frame1/annotations_motility_wiggle/"
annotations_motility_mid = dest_dir + "/frame1/annotations_motility_mid/"

annotations_sticking_motile = dest_dir + "/frame1/annotations_sticking_motile/"
annotations_sticking_non_motile = dest_dir + "/frame1/annotations_sticking_non_motile/"
annotations_sticking_stick = dest_dir + "/frame1/annotations_sticking_stick/"


os.makedirs(images_target, exist_ok=True)
os.makedirs(annotations_easy_hard, exist_ok=True)
os.makedirs(annotations_easy_hard_veryhard, exist_ok=True)
os.makedirs(annotations_hard, exist_ok=True)
os.makedirs(annotations_easy, exist_ok=True)
os.makedirs(annotations_very_hard, exist_ok=True)


os.makedirs(annotations_sticking_stick, exist_ok=True)
os.makedirs(annotations_sticking_motile, exist_ok=True)
os.makedirs(annotations_sticking_non_motile, exist_ok=True)

os.makedirs(annotations_motility_high, exist_ok=True)
os.makedirs(annotations_motility_wiggle, exist_ok=True)
os.makedirs(annotations_motility_mid, exist_ok=True)
os.makedirs(annotations_motility_low, exist_ok=True)

for i,image in enumerate(files):
    #copy and rename images
    image_name = str(count) +".tif"
    shutil.copy(images_source + image, images_target + image_name)
    #create annoatations txt file
    #image_id = int(files_track[i].split(")")[1].split(".tif")[0])
    txt_file = open(annotations_easy_hard + str(count) +".txt",'w')
    txt_file_easy = open(annotations_easy + str(count) +".txt",'w')
    txt_file_hard = open(annotations_hard + str(count) +".txt",'w')
    txt_file_very_hard = open(annotations_very_hard + str(count) +".txt",'w')
    txt_file_all = open(annotations_easy_hard_veryhard + str(count) +".txt",'w')
    
    txt_file_motility_high = open(annotations_motility_high + str(count) +".txt",'w')
    txt_file_motility_low = open(annotations_motility_low + str(count) +".txt",'w')
    txt_file_motility_wiggle = open(annotations_motility_wiggle  + str(count) +".txt",'w')
    txt_file_motility_mid = open(annotations_motility_mid + str(count) +".txt",'w')
    
    txt_file_sticking_stick = open(annotations_sticking_stick + str(count) +".txt",'w')
    txt_file_sticking_motile = open(annotations_sticking_motile + str(count) +".txt",'w')
    txt_file_sticking_non_motile = open(annotations_sticking_non_motile + str(count) +".txt",'w')

    for row in rows:
        # print(image_id)
        # print( row[img_id])
        ##print(row)
        if int(row[img_id])-1 == int(count):#PID starts from 1
            # print(image_id)
            # print( row[img_id])
            txt_file_all.write(row[x_id])
            txt_file_all.write(" ")
            txt_file_all.write(row[y_id])
            txt_file_all.write("\n")
            try:
                if row[diff_id]=="E":
                    txt_file_easy.write(row[x_id])
                    txt_file_easy.write(" ")
                    txt_file_easy.write(row[y_id])
                    txt_file_easy.write("\n")

                    txt_file.write(row[x_id])
                    txt_file.write(" ")
                    txt_file.write(row[y_id])
                    txt_file.write("\n")

                if row[diff_id]=="H":
                    txt_file_hard.write(row[x_id])
                    txt_file_hard.write(" ")
                    txt_file_hard.write(row[y_id])
                    txt_file_hard.write("\n")

                    txt_file.write(row[x_id])
                    txt_file.write(" ")
                    txt_file.write(row[y_id])
                    txt_file.write("\n")                

                if row[diff_id]=="VH":
                    txt_file_very_hard.write(row[x_id])
                    txt_file_very_hard.write(" ")
                    txt_file_very_hard.write(row[y_id])
                    txt_file_very_hard.write("\n")
                    
                if row[subpop_id]=="L":
                    txt_file_motility_low.write(row[x_id])
                    txt_file_motility_low.write(" ")
                    txt_file_motility_low.write(row[y_id])
                    txt_file_motility_low.write("\n")
                if row[subpop_id]=="W":
                   #print("wiggle")
                    txt_file_motility_wiggle.write(row[x_id])
                    txt_file_motility_wiggle.write(" ")
                    txt_file_motility_wiggle.write(row[y_id])
                    txt_file_motility_wiggle.write("\n")
                if row[subpop_id]=="M":
                    txt_file_motility_mid.write(row[x_id])
                    txt_file_motility_mid.write(" ")
                    txt_file_motility_mid.write(row[y_id])
                    txt_file_motility_mid.write("\n")
                if row[subpop_id]=="H":
                    txt_file_motility_high.write(row[x_id])
                    txt_file_motility_high.write(" ")
                    txt_file_motility_high.write(row[y_id])
                    txt_file_motility_high.write("\n")
                
                if row[sticking_id]=="S":
                    txt_file_sticking_stick.write(row[x_id])
                    txt_file_sticking_stick.write(" ")
                    txt_file_sticking_stick.write(row[y_id])
                    txt_file_sticking_stick.write("\n")
                if row[sticking_id]=="M":
                    txt_file_sticking_motile.write(row[x_id])
                    txt_file_sticking_motile.write(" ")
                    txt_file_sticking_motile.write(row[y_id])
                    txt_file_sticking_motile.write("\n")
                if row[sticking_id]=="NM":
                    txt_file_sticking_non_motile.write(row[x_id])
                    txt_file_sticking_non_motile.write(" ")
                    txt_file_sticking_non_motile.write(row[y_id])
                    txt_file_sticking_non_motile.write("\n")
            except:
                txt_file_easy.write(row[x_id])
                txt_file_easy.write(" ")
                txt_file_easy.write(row[y_id])
                txt_file_easy.write("\n")
        
    txt_file.close()
    count = count+1
     
        

Generate Bacteria Tracks specific data for Bacteria Analysis

In [13]:
#run only once to generate bacteria data
def generate_bacteria_data(file, video_dir):
#     all_files = os.listdir(video_data_path)
#     csv_file = list(filter(lambda f: f.endswith('.xlsx'), all_files))[0]
#     #csv_file = "./050421_Agar_sworm_plate#5_Set_1_RBS_2_1_Easy_and_hard.xlsx"
#     read_file = pd.read_excel(os.path.join(video_data_path, csv_file), engine='openpyxl')
#     read_file.to_csv (csv_file.split("xlsx")[0]+".csv", 
#                       index = None,
#                       header=True)

#     file = open(csv_file.split("xlsx")[0]+".csv")
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)
    # header = header[0].split("\t")
    print(header)
    header = [x.lower() for x in header]
    img_id = header.index('pid') #  header.index('t [Frame]') 
    x_id = header.index('x [pixel]')
    y_id = header.index('y [pixel]')
    try:
        diff_id = header.index('e/h')
    except:
        diff_id = None
        print("No diff id found, assuming all easy")
    track_id = header.index('tid')
    print(img_id)
    print(x_id)
    print(y_id)
    print(track_id)

    rows = []
    for row in csvreader:
            rows.append(row)
    print(rows[:6])
    file.close()
    
    tid_visited = []
    #video_dir = "./video9_feature_optical_flow_median_back_2pyr_18win/test/"
    bacteria_folder = "bacteria"
    bacteria_easy_hard_state_file = "easy_hard_veryhard"
    bacteria_coords = "xy_coord"
    count = 0
    max_pid = 0
    prev_tid = 0

    for row in rows:
        pid = int(row[img_id])-1
        if max_pid < pid:
            max_pid = pid

    for row in rows:
        tid = row[track_id]
        if tid not in tid_visited:
            tid_visited.append(tid)

    #         if count<(max_pid-1) and count>0:
    #             # print(row)
    #             # print(rows[i+1])
    #             print(count)
    #             print(max_pid)
    #             for i in range((max_pid - count-1)):
    #                 txt_file.write(str(count))
    #                 txt_file.write(" ")
    #                 txt_file.write("NotPresent")
    #                 txt_file.write("\n")

    #                 coord_file = open(os.path.join(video_dir, bacteria_folder, str(prev_tid), bacteria_coords, str(count)) +".txt",'w')
    #                 coord_file.close()
    #                 count = count+1
            # txt_file.close()
            # coord_file.close()
            count = 0
            os.makedirs(os.path.join(video_dir, bacteria_folder, str(tid), bacteria_coords), exist_ok=True)
            #os.makedirs(os.path.join(video_dir, bacteria_folder, str(tid)), exist_ok=True)
            try:
                os.remove(os.path.join(video_dir, bacteria_folder, str(tid), bacteria_easy_hard_state_file) +".txt")
                #os.remove(os.path.join(video_dir, bacteria_folder, str(tid), bacteria_coords, str(count)) +".txt")
            except OSError:
                pass

        txt_file = open(os.path.join(video_dir, bacteria_folder, str(tid), bacteria_easy_hard_state_file) +".txt",'a')
        pid = int(row[img_id]) - 1
        if int(pid) == 0: #for optical flow since first frame is skipped
            continue
        if pid-2>count: # pid-1 because 1 is skipped
            # print(count)
            # print(pid)
            for i in range((pid - count-1)):
                txt_file.write(str(count))
                txt_file.write(" ")
                txt_file.write("NotPresent")
                txt_file.write("\n")

                coord_file = open(os.path.join(video_dir, bacteria_folder, str(tid), bacteria_coords, str(count)) +".txt",'w')
                coord_file.close()
                count = count+1

        txt_file.write(str(count))
        txt_file.write(" ")
        try:
            txt_file.write(row[diff_id])
        except:
            txt_file.write("E")
            #print("No diff id found, assuming all easy")
        txt_file.write("\n")



        coord_file = open(os.path.join(video_dir, bacteria_folder, str(tid), bacteria_coords, str(count)) +".txt",'a')
        coord_file.write(row[x_id])
        coord_file.write(" ")
        coord_file.write(row[y_id])
        coord_file.write("\n")


        count = count+1

    if count<(max_pid-1) and count>0:
            # print(row)
            # print(rows[i+1])
            print(count)
            print(max_pid)
            for i in range((max_pid - count)):
                txt_file.write(str(count))
                txt_file.write(" ")
                txt_file.write("NotPresent")
                txt_file.write("\n")

                coord_file = open(os.path.join(video_dir, bacteria_folder, str(tid), bacteria_coords, str(count)) +".txt",'w')
                coord_file.close()
                count = count+1
    txt_file.close()
    coord_file.close()

    


In [14]:
target_video_dir = f"{target_data_sub_dir}/video{video_num}/frame1"
file = open(csv_file)
generate_bacteria_data(file, target_video_dir)
file.close()

['Nr', 'TID', 'PID', 'x [pixel]', 'y [pixel]', 't [sec]', 'I [val]', 'Len [pixel]', 'D2S [pixel]', 'D2R [pixel]', 'D2P [pixel]', 'v [pixel/sec]', '? [deg]', '?? [deg]', 'E/H', 'DEL_Diff', 'Sticking', 'Subpopulation']
2
3
4
1
[['1', '1', '1', '461.757', '347.733', '0', '196', '0.0', '0.0', '', '', '', '', '', 'H', '-0.8828949319260454', 'M', 'H'], ['2', '1', '2', '461.757', '347.733', '0', '194', '0.0', '0.0', '', '0.0', '', '0.0', '', 'H', '0.2888720548427962', 'M', 'H'], ['3', '1', '3', '461.757', '347.733', '0', '191', '0.0', '0.0', '', '0.0', '', '0.0', '0.0', 'H', '0.4209067180548305', 'M', 'H'], ['4', '1', '4', '462.784', '347.733', '0', '217', '1.027', '1.027', '', '1.027', '', '0.0', '0.0', 'H', '0.5165881079910245', 'M', 'H'], ['5', '1', '5', '462.784', '347.733', '0', '187', '1.027', '1.027', '', '0.0', '', '0.0', '0.0', 'H', '0.5323234989267642', 'M', 'H'], ['6', '1', '6', '462.784', '347.733', '0', '186', '1.027', '1.027', '', '0.0', '', '0.0', '0.0', 'H', '0.529890554700802

In [15]:
#delete the unzipped folder in main data dir
shutil.rmtree(video_dir)

create feature data using motion features

In [16]:
def create_video(data_dir):
    # choose codec according to format needed
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    #print(data_dir)
    img_sample = cv2.imread(os.path.join(data_dir,"images/0.tif"))
    #print(img_sample.shape)
    height, width, channels = img_sample.shape
    
    video = cv2.VideoWriter(data_dir + 'video.mp4', fourcc, 1, (width, height))
    #data_dir = "./Data/video3/"
    image_dir = os.path.join(data_dir, "images")
    for frame in natsorted(os.listdir(image_dir)):
        #print(frame)
        img = cv2.imread(os.path.join(image_dir, frame))
        video.write(img)

    cv2.destroyAllWindows()
    video.release()

In [17]:
#iterate through video1,2,4
#create videos for all frame sets- 1-1,1-2..2-1..4-2
data_path = target_data_sub_dir
test_video = [f"video{video_num}"]
for video in natsorted(test_video):
    if not video.startswith('.') and os.path.isdir(os.path.join(data_path,video))==True:
        for minivideo in natsorted(os.listdir(os.path.join(data_path,video))):
            if not minivideo.startswith('.') and os.path.isdir(os.path.join(data_path,video,minivideo)) == True:
                #print(minivideo)
                create_video(os.path.join(data_path,video,minivideo))
    
    

In [18]:
def get_background(file_path, mean=True, sample=False):
    cap = cv2.VideoCapture(file_path)
    #print(cap.read())
    # we will randomly select 50 frames for the calculating the median
    #frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=20)
    frame_indices = list(range(0,int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) ))
    print(len(frame_indices))
    # we will store the frames in array
    frames = []
    for idx in frame_indices:
        # set the frame id to read that particular frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        #print(ret)
        frames.append(frame)
    if mean:
         # calculate the mean
        background_frame = np.mean(frames, axis=0).astype(np.uint8)
    else:
        # calculate the median
        background_frame = np.median(frames, axis=0).astype(np.uint8)
    if sample==True:
        background_frame = cv2.imread("./Control_2_b0t5306c0x0-660y0-492.tiff")
        #background_frame = cv2.imread("./RBS 2_1_b0t2791c0x0-660y0-492.tiff")
    return background_frame

In [19]:
get_background(f"{target_data_sub_dir}/video{video_num}/frame1video.mp4").shape

150


(492, 660, 3)